# HEC-RAS 2D HDF Data Analysis Notebook

This notebook demonstrates how to manipulate and analyze HEC-RAS 2D HDF data using the ras-commander library. It leverages the RasHdf and RasUtils classes to streamline data extraction, processing, and visualization.

In [1]:
# Import required Libraries
import subprocess
import sys
import os
from pathlib import Path

def install_module(module_name):
    try:
        __import__(module_name)
    except ImportError:
        print(f"{module_name} not found. Installing...")
        subprocess.check_call([sys.executable, "-m", "pip", "install", "-U", module_name])

# List of modules to check and install if necessary
modules = ['h5py', 'numpy', 'requests', 'geopandas', 'matplotlib', 'pandas']
for module in modules:
    install_module(module)

# Import the rest of the required libraries
import pandas as pd
import numpy as np
import geopandas as gpd
import matplotlib.pyplot as plt


## Importing ras-commander flexibly (from package or local dev copy)

In [2]:
import sys
from pathlib import Path

# Flexible imports to allow for development without installation 
#  ** Use this version with Jupyter Notebooks **
try:
    # Try to import from the installed package
    from ras_commander import init_ras_project, RasHdf, RasExamples, RasCmdr, RasPlan, RasGeo, RasUnsteady, RasUtils, RasPrj, RasGpt, ras
except ImportError:
    # If the import fails, add the parent directory to the Python path
    import os
    current_file = Path(os.getcwd()).resolve()
    parent_directory = current_file.parent
    sys.path.append(str(parent_directory))
    
    # Now try to import again
    from ras_commander import init_ras_project, RasHdf, RasExamples, RasCmdr, RasPlan, RasGeo, RasUnsteady, RasUtils, RasPrj, RasGpt, ras

print("ras_commander imported successfully")


ras_commander imported successfully


In [3]:
# Define the path to the Muncie project
current_dir = Path.cwd()  # Adjust if your notebook is in a different directory
muncie_path = current_dir / "example_projects" / "Muncie"
bald_eagle_path = current_dir / "example_projects" / "BaldEagleCrkMulti2D"
import logging
# Check if Muncie.p03.hdf exists (so we don't have to re-run the simulation when re-running or debugging)
hdf_file = muncie_path / "Muncie.p03.hdf"

if not hdf_file.exists():
    # Initialize RasExamples and extract the Muncie project
    ras_examples = RasExamples()
    ras_examples.extract_project(["Muncie", "BaldEagleCrkMulti2D"])

    # Initialize custom Ras objects
    muncie = RasPrj()
    bald_eagle = RasPrj()

    # Initialize the RAS projects using the custom ras objects
    muncie = init_ras_project(muncie_path, "6.6", ras_instance=muncie)
    logging.info(f"Muncie project initialized with folder: {muncie.project_folder}")

    bald_eagle = init_ras_project(bald_eagle_path, "6.6", ras_instance=bald_eagle)
    logging.info(f"Bald Eagle project initialized with folder: {bald_eagle.project_folder}")
    
    logging.info(f"Muncie object id: {id(muncie)}")
    logging.info(f"Bald Eagle object id: {id(bald_eagle)}")
    
    # Define the plan number to execute
    plan_number = "03"

    # Set plan keys for both projects
    for project in [muncie, bald_eagle]:
        RasPlan.update_plan_value(plan_number, "run_htab", 1, ras_object=project)
        RasPlan.update_plan_value(plan_number, "run_unet", 1, ras_object=project)
        RasPlan.update_plan_value(plan_number, "run_postProcess", 1, ras_object=project)
        RasPlan.update_plan_value(plan_number, "run_rasmapper", 0, ras_object=project)

    # Execute Plan 03 using RasCmdr for Muncie
    print(f"Executing Plan {plan_number} for the Muncie project...")
    success_muncie = RasCmdr.compute_plan(plan_number, ras_object=muncie)
    if success_muncie:
        print(f"Plan {plan_number} executed successfully for Muncie.\n")
    else:
        print(f"Plan {plan_number} execution failed for Muncie.\n")
    
    # Execute Plan 03 using RasCmdr for Bald Eagle
    print(f"Executing Plan {plan_number} for the Bald Eagle Creek project...")
    success_bald_eagle = RasCmdr.compute_plan(plan_number, ras_object=bald_eagle)
    if success_bald_eagle:
        print(f"Plan {plan_number} executed successfully for Bald Eagle.\n")
    else:
        print(f"Plan {plan_number} execution failed for Bald Eagle.\n")
else:
    print("Muncie.p03.hdf already exists. Skipping project extraction and plan execution.")
    # Initialize the RAS project using the custom ras object
    muncie = RasPrj()
    bald_eagle = RasPrj()
    muncie = init_ras_project(muncie_path, "6.6", ras_instance=muncie)
    bald_eagle = init_ras_project(bald_eagle_path, "6.6", ras_instance=bald_eagle)
    plan_number = "03"

2024-10-07 17:57:33,226 - ras_commander.RasPrj - INFO - HEC-RAS executable found at default path: C:\Program Files (x86)\HEC\HEC-RAS\6.6\Ras.exe
2024-10-07 17:57:33,241 - ras_commander.RasPrj - INFO - Initialization complete for project: Muncie
2024-10-07 17:57:33,241 - ras_commander.RasPrj - INFO - Plan entries: 3, Flow entries: 1, Unsteady entries: 1, Geometry entries: 3, Boundary conditions: 2
2024-10-07 17:57:33,242 - ras_commander.RasPrj - INFO - Project initialized. ras_instance project folder: c:\GH\ras-commander\examples\example_projects\Muncie
2024-10-07 17:57:33,244 - ras_commander.RasPrj - INFO - HEC-RAS executable found at default path: C:\Program Files (x86)\HEC\HEC-RAS\6.6\Ras.exe
2024-10-07 17:57:33,271 - ras_commander.RasPrj - INFO - Initialization complete for project: BaldEagleDamBrk
2024-10-07 17:57:33,272 - ras_commander.RasPrj - INFO - Plan entries: 11, Flow entries: 0, Unsteady entries: 10, Geometry entries: 10, Boundary conditions: 51
2024-10-07 17:57:33,273 - ra

Muncie.p03.hdf already exists. Skipping project extraction and plan execution.


In [4]:
# Retrieve the HDF file path for Plan 03
results_path_muncie = RasPlan.get_results_path(plan_number, ras_object=muncie)
if results_path_muncie:
    print(f"Results for Plan {plan_number} are located at: {results_path_muncie}\n")
else:
    print(f"No results found for Plan {plan_number}.\n")
    
results_path_baldeagle = RasPlan.get_results_path(plan_number, ras_object=bald_eagle)
if results_path_baldeagle:
    print(f"Results for Plan {plan_number} are located at: {results_path_baldeagle}\n")
else:
    print(f"No results found for Plan {plan_number}.\n")    

Results for Plan 03 are located at: c:\GH\ras-commander\examples\example_projects\Muncie\Muncie.p03.hdf

Results for Plan 03 are located at: c:\GH\ras-commander\examples\example_projects\BaldEagleCrkMulti2D\BaldEagleDamBrk.p03.hdf



In [5]:
# Define the HDF input path as Plan Number
hdf_input = plan_number

# Initialize RasHdf handler
hdf_handler = RasHdf()

# The remainder of the examples only use muncie, so let's set results_path to muncie
results_path = results_path_muncie



In [6]:
# Example 1: List all HDF paths with properties
print("Example 1: Listing all HDF paths with properties")
hdf_paths_df = RasHdf.get_hdf_paths_with_properties(hdf_input, ras_object=muncie)
display(hdf_paths_df.head())

Example 1: Listing all HDF paths with properties


,HDF_Path,Type,Shape,Size,Dtype
0,Event Conditions,Group,None,NaN,None
1,Event Conditions/Unsteady,Group,None,NaN,None
2,Event Conditions/Unsteady/Boundary Conditions,Group,None,NaN,None
3,Event Conditions/Unsteady/Boundary Conditions/...,Group,None,NaN,None
4,Event Conditions/Unsteady/Boundary Conditions/...,Dataset,"(25, 2)",50.0,float32


In [7]:
# Example 2: Extract runtime and compute time data
print("\nExample 2: Extracting runtime and compute time data")
runtime_df = RasHdf.get_runtime_data(hdf_input, ras_object=muncie)
if runtime_df is not None:
    display(runtime_df)
else:
    print("No runtime data found.")

2024-10-07 17:57:33,511 - ras_commander.RasHdf - INFO - Extracting Plan Information from: Muncie.p03.hdf
2024-10-07 17:57:33,513 - ras_commander.RasHdf - INFO - Plan Name: Unsteady Run with 2D 50ft Grid
2024-10-07 17:57:33,515 - ras_commander.RasHdf - INFO - Simulation Start Time: 02Jan1900 00:00:00
2024-10-07 17:57:33,516 - ras_commander.RasHdf - INFO - Simulation End Time: 03Jan1900 00:00:00
2024-10-07 17:57:33,516 - ras_commander.RasHdf - INFO - Simulation Duration (hours): 24.0



Example 2: Extracting runtime and compute time data


,Plan Name,File Name,Simulation Start Time,Simulation End Time,Simulation Duration (s),Simulation Time (hr),Completing Geometry (hr),Preprocessing Geometry (hr),Completing Event Conditions (hr),Unsteady Flow Computations (hr),Complete Process (hr),Unsteady Flow Speed (hr/hr),Complete Process Speed (hr/hr)
0,Unsteady Run with 2D 50ft Grid,Muncie.p03.hdf,02Jan1900 00:00:00,03Jan1900 00:00:00,86400.0,24.0,N/A,0.000373,N/A,0.011337,0.014089,2116.972533,1703.503618


In [8]:
# Example 3: Get 2D Flow Area Names
print("\nExample 3: Listing 2D Flow Area Names")
flow_area_names = RasHdf.get_2d_flow_area_names(hdf_input, ras_object=muncie)
print("2D Flow Area Names:", flow_area_names)

2024-10-07 17:57:33,547 - ras_commander.RasHdf - INFO - Found 1 2D Flow Areas



Example 3: Listing 2D Flow Area Names
2D Flow Area Names: ['2D Interior Area']


In [9]:
# Example 4: Extract 2D Flow Area Attributes
print("\nExample 4: Extracting 2D Flow Area Attributes")
flow_area_attributes_df = RasHdf.get_2d_flow_area_attributes(hdf_input, ras_object=muncie)
if flow_area_attributes_df is not None:
    display(flow_area_attributes_df.head())
else:
    print("No 2D Flow Area attributes found.")


Example 4: Extracting 2D Flow Area Attributes


,Name,Locked,Mann,Multiple Face Mann n,Composite LC,Cell Vol Tol,Cell Min Area Fraction,Face Profile Tol,Face Area Tol,Face Conv Ratio,Laminar Depth,Min Face Length Ratio,Spacing dx,Spacing dy,Shift dx,Shift dy,Cell Count
0,b'2D Interior Area',0,0.06,0,0,0.01,0.01,0.01,0.01,0.02,0.2,0.05,50.0,50.0,NaN,NaN,5391


In [10]:
# Example 5: Extract Cell Info
print("\nExample 5: Extracting Cell Info")
cell_info_df = RasHdf.get_cell_info(hdf_input, ras_object=muncie)
if cell_info_df is not None:
    display(cell_info_df.head())
else:
    print("No Cell Info found.")


Example 5: Extracting Cell Info


,Start,End
0,0,5391


In [11]:
# Example 6: Extract Cell Points
print("\nExample 6: Extracting Cell Points")
cell_points_df = RasHdf.get_cell_points(hdf_input, ras_object=muncie)
if cell_points_df is not None:
    display(cell_points_df.head())
else:
    print("No Cell Points found.")


Example 6: Extracting Cell Points


,X,Y
0,406000.0,1805000.0
1,406050.0,1805000.0
2,406100.0,1805000.0
3,406150.0,1805000.0
4,406200.0,1805000.0


In [12]:
# Example 7: Extract Polygon Info and Parts
print("\nExample 7: Extracting Polygon Info and Parts")
polygon_info_df, polygon_parts_df = RasHdf.get_polygon_info_and_parts(hdf_input, ras_object=muncie)
print("Polygon Info:")
if polygon_info_df is not None:
    display(polygon_info_df.head())
else:
    print("No Polygon Info found.")
print("\nPolygon Parts:")
if polygon_parts_df is not None:
    display(polygon_parts_df.head())
else:
    print("No Polygon Parts found.")


Example 7: Extracting Polygon Info and Parts
Polygon Info:


,Column1,Column2,Column3,Column4
0,0,170,0,1



Polygon Parts:


,Start,Count
0,0,170


In [13]:
# Example 8: Extract Polygon Points
print("\nExample 8: Extracting Polygon Points")
polygon_points_df = RasHdf.get_polygon_points(hdf_input, ras_object=muncie)
if polygon_points_df is not None:
    display(polygon_points_df.head())
else:
    print("No Polygon Points found.")


Example 8: Extracting Polygon Points


,X,Y
0,409537.1821,1802597.314
1,409645.1430,1802591.258
2,409737.8005,1802600.670
3,409762.3667,1802605.741
4,409846.7495,1802629.917


In [14]:
# Example 9: Extract Cells Center Coordinates and Manning's n
print("\nExample 9: Extracting Cells Center Coordinates and Manning's n")
cells_center_coord_df, cells_manning_n_df = RasHdf.get_cells_center_data(hdf_input, ras_object=muncie)
print("Cells Center Coordinates:")
if cells_center_coord_df is not None:
    display(cells_center_coord_df.head())
else:
    print("No Cells Center Coordinates found.")
print("\nCells Manning's n:")
if cells_manning_n_df is not None:
    display(cells_manning_n_df.head())
else:
    print("No Cells Manning's n found.")


Example 9: Extracting Cells Center Coordinates and Manning's n
Cells Center Coordinates:


,X,Y
0,406000.0,1805000.0
1,406050.0,1805000.0
2,406100.0,1805000.0
3,406150.0,1805000.0
4,406200.0,1805000.0



Cells Manning's n:


,Manning's n
0,0.06
1,0.06
2,0.06
3,0.06
4,0.06


In [15]:
# Example 10: Extract Faces Area Elevation Data
print("\nExample 10: Extracting Faces Area Elevation Data")
faces_elev_df = RasHdf.get_faces_area_elevation_data(hdf_input, ras_object=muncie)
if faces_elev_df is not None:
    display(faces_elev_df.head())
else:
    print("No Faces Area Elevation Data found.")


Example 10: Extracting Faces Area Elevation Data


,Elevation,Area,Wetted Perimeter,Manning's n
0,939.428467,0.000000,0.000000,0.06
1,939.628479,0.314608,3.152243,0.06
2,939.756226,0.838549,5.068435,0.06
3,939.760376,0.859697,5.130248,0.06
4,939.979004,2.336330,8.406319,0.06


In [16]:
# Example 11: Extract Compute Messages as String
print("\nExample 11: Extracting Compute Messages")
compute_messages = RasHdf.extract_string_from_hdf(hdf_input, '/Results/Summary/Compute Messages (text)', ras_object=muncie)
print("Compute Messages:")
print(compute_messages)


Example 11: Extracting Compute Messages
Compute Messages:
['Plan: \'Unsteady Run with 2D 50ft Grid\' (Muncie.p03)\r\nSimulation started at: 07Oct2024 05:39:28 PM\r\n\r\nWriting Plan GIS Data...\r\nCompleted Writing Plan GIS Data\r\nWriting Geometry...\r\nComputing Bank Lines\r\nBank lines generated in 99 ms\r\nComputing Edge Lines\r\nEdge Lines generated in 64 ms\r\nComputing XS Interpolation Surface\r\nXS Interpolation Surface generated in 120 ms\r\nGeometry \'Geometry\' association was set to the first valid terrain layer (Terrain)\r\nComputing 2D Flow Area \'2D Interior Area\' tables: Property tables do not exist.\r\n2D Flow Area \'2D Interior Area\' tables complete 3.08 sec\r\nCompleted Writing Geometry\r\nWriting Event Conditions ...\r\nCompleted Writing Event Condition Data\r\n\r\n\t\r\nGeometric Preprocessor HEC-RAS 6.6 September 2024\r\n \r\n\r\nFinished Processing Geometry\r\n\r\n\r\nPerforming Unsteady Flow Simulation  HEC-RAS 6.6 September 2024\r\n \r\n\t\r\nUnsteady Input 

In [17]:
# Example 12: Extract Plan Parameters and Volume Accounting
print("\nExample 12: Extracting Plan Parameters and Volume Accounting Data")
import h5py
import numpy as np

# Extract plan parameters
with h5py.File(str(results_path), 'r') as hdf_file:
    plan_parameters = hdf_file['Plan Data/Plan Parameters']
    
    # List group attributes
    print("Plan Parameters Group Attributes:")
    #for attr_name, attr_value in plan_parameters.attrs.items():
        #print(f"{attr_name}: {attr_value}")
    
    # Extract plan parameters as a DataFrame
    plan_parameters_df = pd.DataFrame([(attr_name, attr_value) for attr_name, attr_value in plan_parameters.attrs.items()], columns=['Attribute', 'Value'])

# Construct the group path for volume accounting data
group_to_list = "Results/Unsteady/Summary/Volume Accounting/Volume Accounting 2D"

# Extract volume accounting data as a DataFrame
volume_accounting_df = RasHdf.get_group_attributes_as_df(results_path, group_to_list)

print("\nPlan Parameters DataFrame:")
display(plan_parameters_df)

print("\nVolume Accounting DataFrame:")
display(volume_accounting_df)


Example 12: Extracting Plan Parameters and Volume Accounting Data
Plan Parameters Group Attributes:

Plan Parameters DataFrame:


,Attribute,Value
0,1D Cores,0
1,1D Flow Tolerance,NaN
2,1D Maximum Iterations,20
3,1D Maximum Iterations Without Improvement,0
4,1D Maximum Water Surface Error To Abort,100.0
5,1D Methodology,b'Finite Difference'
6,1D Storage Area Elevation Tolerance,0.01
7,1D Theta,1.0
8,1D Theta Warmup,1.0
9,1D Water Surface Elevation Tolerance,0.01



Volume Accounting DataFrame:


,Attribute,Value,Type,Shape,Size,Dtype
0,Vol Accounting in,Acre Feet,bytes_,None,None,None


In [18]:
# Example 13: Listing 2D Flow Area Groups
print("\nExample: Listing 2D Flow Area Groups")

# Get the names of all 2D Flow Area groups
flow_area_group_names = RasHdf.get_2d_flow_area_names(hdf_input, ras_object=muncie)

print("2D Flow Area Groups:")
if flow_area_group_names:
    for name in flow_area_group_names:
        print(f"- {name}")
else:
    print("No 2D Flow Area groups found in the HDF file.")

import h5py

# Additional information about the first group (if any)
if flow_area_group_names:
    first_group = flow_area_group_names[0]
    print(f"\nDetailed information for the first group: {first_group}")
    
    # Remember, use results_path because we are accessing the hdf file and hdf_input is the plan number
    with h5py.File(str(results_path), 'r') as hdf_file:
        group = hdf_file[f'Geometry/2D Flow Areas/{first_group}']
        print("Datasets in this group:")
        for name, item in group.items():
            if isinstance(item, h5py.Dataset):
                print(f"- {name}: Shape {item.shape}, Dtype {item.dtype}")


Example: Listing 2D Flow Area Groups


2024-10-07 17:57:33,847 - ras_commander.RasHdf - INFO - Found 1 2D Flow Areas


2D Flow Area Groups:
- 2D Interior Area

Detailed information for the first group: 2D Interior Area
Datasets in this group:
- Cells Center Coordinate: Shape (5765, 2), Dtype float64
- Cells Center Manning's n: Shape (5765,), Dtype float32
- Cells Face and Orientation Info: Shape (5765, 2), Dtype int32
- Cells Face and Orientation Values: Shape (22328, 2), Dtype int32
- Cells FacePoint Indexes: Shape (5765, 7), Dtype int32
- Cells Minimum Elevation: Shape (5765,), Dtype float32
- Cells Surface Area: Shape (5765,), Dtype float32
- Cells Volume Elevation Info: Shape (5765, 2), Dtype int32
- Cells Volume Elevation Values: Shape (37912, 2), Dtype float32
- FacePoints Cell Index Values: Shape (22702,), Dtype int32
- FacePoints Cell Info: Shape (5774, 2), Dtype int32
- FacePoints Coordinate: Shape (5774, 2), Dtype float64
- FacePoints Face and Orientation Info: Shape (5774, 2), Dtype int32
- FacePoints Face and Orientation Values: Shape (22328, 2), Dtype int32
- FacePoints Is Perimeter: Shape

In [19]:
# Example 14: Extract Faces Indexes
print("\nExample 13: Extracting Faces Indexes")
cell_indexes_df, facepoint_indexes_df = RasHdf.get_faces_indexes(hdf_input, ras_object=muncie)
print("Faces Cell Indexes:")
if cell_indexes_df is not None:
    display(cell_indexes_df.head())
else:
    print("No Faces Cell Indexes found.")
print("\nFaces FacePoint Indexes:")
if facepoint_indexes_df is not None:
    display(facepoint_indexes_df.head())
else:
    print("No Faces FacePoint Indexes found.")


Example 13: Extracting Faces Indexes
Faces Cell Indexes:


,Left Cell,Right Cell
0,1,2
1,1,5391
2,1,0
3,1,13
4,2,14



Faces FacePoint Indexes:


,Start FacePoint,End FacePoint
0,4,5
1,5,1
2,1,0
3,0,4
4,4,6


In [20]:
# Example 15: Extract Faces Elevation Data
print("\nExample 14: Extracting Faces Elevation Data")
low_elev_centroid_df, min_elevation_df = RasHdf.get_faces_elevation_data(hdf_input, ras_object=muncie)
print("Faces Low Elevation Centroid:")
if low_elev_centroid_df is not None:
    display(low_elev_centroid_df.head())
else:
    print("No Faces Low Elevation Centroid found.")
print("\nFaces Minimum Elevation:")
if min_elevation_df is not None:
    display(min_elevation_df.head())
else:
    print("No Faces Minimum Elevation found.")


Example 14: Extracting Faces Elevation Data
Faces Low Elevation Centroid:


,Low Elevation Centroid
0,1.450514
1,18.177126
2,36.361233
3,44.354172
4,37.621490



Faces Minimum Elevation:


,Minimum Elevation
0,939.428467
1,940.156250
2,940.156250
3,939.428467
4,939.091248


In [21]:
# Example 16: Extract Faces Vector Data
print("\nExample 15: Extracting Faces Vector Data")
faces_vector_df = RasHdf.get_faces_vector_data(hdf_input, ras_object=muncie)
if faces_vector_df is not None:
    display(faces_vector_df.head())
else:
    print("No Faces Vector Data found.")


Example 15: Extracting Faces Vector Data


,NormalX,NormalY,Length
0,1.00000,-0.000000,43.544975
1,-0.06601,0.997819,50.109291
2,-1.00000,-0.000000,40.237263
3,-0.00000,-1.000000,50.000000
4,-0.00000,-1.000000,50.000000


In [22]:
# Example 17: Extract Faces Perimeter Data
print("\nExample 16: Extracting Faces Perimeter Data")
perimeter_info_df, perimeter_values_df = RasHdf.get_faces_perimeter_data(hdf_input, ras_object=muncie)
print("Faces Perimeter Info:")
if perimeter_info_df is not None:
    display(perimeter_info_df.head())
else:
    print("No Faces Perimeter Info found.")
print("\nFaces Perimeter Values:")
if perimeter_values_df is not None:
    display(perimeter_values_df.head())
else:
    print("No Faces Perimeter Values found.")


Example 16: Extracting Faces Perimeter Data
Faces Perimeter Info:


,Start,Count
0,0,0
1,0,0
2,0,0
3,0,0
4,0,0



Faces Perimeter Values:


,X,Y
0,407715.3687,1804189.615
1,407784.1551,1804114.051
2,407778.0565,1804118.342
3,407773.5300,1804118.765
4,406001.1427,1805013.659


In [23]:
# Example 18: Extract Infiltration Data
print("\nExample 17: Extracting Infiltration Data")
cell_classifications_df, face_classifications_df, initial_deficit_df, maximum_deficit_df, potential_percolation_rate_df = RasHdf.get_infiltration_data(hdf_input, ras_object=muncie)
print("Infiltration - Cell Classifications:")
if cell_classifications_df is not None:
    display(cell_classifications_df.head())
else:
    print("No Infiltration Cell Classifications found.")
print("\nInfiltration - Face Classifications:")
if face_classifications_df is not None:
    display(face_classifications_df.head())
else:
    print("No Infiltration Face Classifications found.")
print("\nInfiltration - Initial Deficit:")
if initial_deficit_df is not None:
    display(initial_deficit_df.head())
else:
    print("No Infiltration Initial Deficit found.")
print("\nInfiltration - Maximum Deficit:")
if maximum_deficit_df is not None:
    display(maximum_deficit_df.head())
else:
    print("No Infiltration Maximum Deficit found.")
print("\nInfiltration - Potential Percolation Rate:")
if potential_percolation_rate_df is not None:
    display(potential_percolation_rate_df.head())
else:
    print("No Infiltration Potential Percolation Rate found.")


Example 17: Extracting Infiltration Data
Infiltration - Cell Classifications:
No Infiltration Cell Classifications found.

Infiltration - Face Classifications:
No Infiltration Face Classifications found.

Infiltration - Initial Deficit:
No Infiltration Initial Deficit found.

Infiltration - Maximum Deficit:
No Infiltration Maximum Deficit found.

Infiltration - Potential Percolation Rate:
No Infiltration Potential Percolation Rate found.


In [24]:
# Example 19: Extract Percent Impervious Data
print("\nExample 18: Extracting Percent Impervious Data")
cell_classifications_df, face_classifications_df, percent_impervious_df = RasHdf.get_percent_impervious_data(hdf_input, ras_object=muncie)
print("Percent Impervious - Cell Classifications:")
if cell_classifications_df is not None:
    display(cell_classifications_df.head())
else:
    print("No Percent Impervious Cell Classifications found.")
print("\nPercent Impervious - Face Classifications:")
if face_classifications_df is not None:
    display(face_classifications_df.head())
else:
    print("No Percent Impervious Face Classifications found.")
print("\nPercent Impervious:")
if percent_impervious_df is not None:
    display(percent_impervious_df.head())
else:
    print("No Percent Impervious data found.")
    
    # Note: Does not exist in the Muncie Plan 3 example used. 


Example 18: Extracting Percent Impervious Data
Percent Impervious - Cell Classifications:
No Percent Impervious Cell Classifications found.

Percent Impervious - Face Classifications:
No Percent Impervious Face Classifications found.

Percent Impervious:
No Percent Impervious data found.


In [25]:
# Example 20: Extract Perimeter Data
print("\nExample 19: Extracting Perimeter Data")
perimeter_df = RasHdf.get_perimeter_data(hdf_input, ras_object=muncie)
print("Perimeter Data:")
if perimeter_df is not None:
    display(perimeter_df.head())
else:
    print("No Perimeter Data found.")


Example 19: Extracting Perimeter Data
Perimeter Data:


,X,Y
0,409537.1821,1802597.314
1,409426.1403,1802614.326
2,409315.2430,1802638.443
3,409133.8857,1802742.001
4,409053.8968,1802841.512


In [26]:
# Example 21: Extract Boundary Condition Lines Attributes
print("\nExample 20: Extracting Boundary Condition Lines Attributes")
bc_lines_df = RasHdf.get_group_attributes_as_df(hdf_input, '/Geometry/Boundary Condition Lines/Attributes', ras_object=muncie)
if bc_lines_df is not None:
    display(bc_lines_df.head())
else:
    print("No Boundary Condition Lines Attributes found.")
    
# None in Muncie plan 3, no output expected.

2024-10-07 17:57:34,060 - ras_commander.RasHdf - CRITICAL - Group path '/Geometry/Boundary Condition Lines/Attributes' not found in HDF file 'c:\GH\ras-commander\examples\example_projects\Muncie\Muncie.p03.hdf'



Example 20: Extracting Boundary Condition Lines Attributes
No Boundary Condition Lines Attributes found.


In [27]:
# Example 22: Extract Boundary Condition Time Series Data
print("\nExample 22: Extracting Boundary Condition Time Series Data")
bc_time_series_df = RasHdf.get_group_attributes_as_df(hdf_input, '/Geometry/Boundary Condition Lines/Attributes', ras_object=muncie)
if bc_time_series_df is not None:
    display(bc_time_series_df.head())
else:
    print("No Boundary Condition Time Series Data found.")

print("\nAll RasHdf functions have been executed on Plan 03 HDF file.")


# NOTE: Muncie does not have boundary conditions in the HDF file - see how errors are handled. 


Example 22: Extracting Boundary Condition Time Series Data


2024-10-07 17:57:34,077 - ras_commander.RasHdf - CRITICAL - Group path '/Geometry/Boundary Condition Lines/Attributes' not found in HDF file 'c:\GH\ras-commander\examples\example_projects\Muncie\Muncie.p03.hdf'


No Boundary Condition Time Series Data found.

All RasHdf functions have been executed on Plan 03 HDF file.


In [28]:
# Example 23: Retrieving 2D Flow Area Solution Times
print("Example 23: Retrieving 2D Flow Area Solution Times")

solution_times = RasHdf.get_2d_area_solution_times(hdf_input, ras_object=muncie)

if solution_times is not None:
    print(f"Retrieved {len(solution_times)} solution times:")
    print(solution_times)
else:
    print("No solution times found.")


Example 23: Retrieving 2D Flow Area Solution Times
Retrieved 289 solution times:
[0.         0.00347222 0.00694444 0.01041667 0.01388889 0.01736111
 0.02083333 0.02430556 0.02777778 0.03125    0.03472222 0.03819444
 0.04166667 0.04513889 0.04861111 0.05208333 0.05555556 0.05902778
 0.0625     0.06597222 0.06944444 0.07291667 0.07638889 0.07986111
 0.08333333 0.08680556 0.09027778 0.09375    0.09722222 0.10069444
 0.10416667 0.10763889 0.11111111 0.11458333 0.11805556 0.12152778
 0.125      0.12847222 0.13194444 0.13541667 0.13888889 0.14236111
 0.14583333 0.14930556 0.15277778 0.15625    0.15972222 0.16319444
 0.16666667 0.17013889 0.17361111 0.17708333 0.18055556 0.18402778
 0.1875     0.19097222 0.19444444 0.19791667 0.20138889 0.20486111
 0.20833333 0.21180556 0.21527778 0.21875    0.22222222 0.22569444
 0.22916667 0.23263889 0.23611111 0.23958333 0.24305556 0.24652778
 0.25       0.25347222 0.25694444 0.26041667 0.26388889 0.26736111
 0.27083333 0.27430556 0.27777778 0.28125    0.2

In [29]:
# Example 24: Retrieving 2D Flow Area Solution Time Dates
print("\nExample 24: Retrieving 2D Flow Area Solution Time Dates")

solution_time_dates = RasHdf.get_2d_area_solution_time_dates(hdf_input, ras_object=muncie)

if solution_time_dates is not None:
    print(f"Retrieved {len(solution_time_dates)} solution time dates:")
    print(solution_time_dates)
else:
    print("No solution time dates found.")



Example 24: Retrieving 2D Flow Area Solution Time Dates
Retrieved 289 solution time dates:
[b'02JAN1900 00:00:00' b'02JAN1900 00:05:00' b'02JAN1900 00:10:00'
 b'02JAN1900 00:15:00' b'02JAN1900 00:20:00' b'02JAN1900 00:25:00'
 b'02JAN1900 00:30:00' b'02JAN1900 00:35:00' b'02JAN1900 00:40:00'
 b'02JAN1900 00:45:00' b'02JAN1900 00:50:00' b'02JAN1900 00:55:00'
 b'02JAN1900 01:00:00' b'02JAN1900 01:05:00' b'02JAN1900 01:10:00'
 b'02JAN1900 01:15:00' b'02JAN1900 01:20:00' b'02JAN1900 01:25:00'
 b'02JAN1900 01:30:00' b'02JAN1900 01:35:00' b'02JAN1900 01:40:00'
 b'02JAN1900 01:45:00' b'02JAN1900 01:50:00' b'02JAN1900 01:55:00'
 b'02JAN1900 02:00:00' b'02JAN1900 02:05:00' b'02JAN1900 02:10:00'
 b'02JAN1900 02:15:00' b'02JAN1900 02:20:00' b'02JAN1900 02:25:00'
 b'02JAN1900 02:30:00' b'02JAN1900 02:35:00' b'02JAN1900 02:40:00'
 b'02JAN1900 02:45:00' b'02JAN1900 02:50:00' b'02JAN1900 02:55:00'
 b'02JAN1900 03:00:00' b'02JAN1900 03:05:00' b'02JAN1900 03:10:00'
 b'02JAN1900 03:15:00' b'02JAN1900 03

In [30]:
# Example 25: Loading 2D Flow Area Solutions
print("\nExample 25: Loading 2D Flow Area Solutions")

# Load 2D area solutions using the revised method
solutions = RasHdf.load_2d_area_solutions(hdf_input, ras_object=muncie)

# Check if solutions were successfully loaded
if solutions:
    # Access and display solution times
    solution_times_df = solutions.get('solution_times')
    if solution_times_df is not None:
        print("Solution Times:")
        print(solution_times_df.head())
    else:
        print("Solution times not found.")
    
    # Iterate through each 2D Flow Area and display sample data
    for key in solutions:
        if key != 'solution_times':
            print(f"\nData for {key}:")
            area_df = solutions[key]
            print(area_df.head(10))  # Display first 10 records
else:
    print("No solutions loaded.")



Example 25: Loading 2D Flow Area Solutions
No solutions loaded.


In [31]:
# Example 29: Building Face FacePoints
print("\nExample 29: Building Face FacePoints")

face_facepoints_list = RasHdf.build_face_facepoints(hdf_input, ras_object=muncie)

if face_facepoints_list:
    # Displaying face points indexes for the first face of the first 2D Flow Area
    print("FacePoints Indexes for the first face of the first 2D Flow Area:")
    print(face_facepoints_list[0][0])  # Assuming at least one area and one face
else:
    print("No face facepoints built.")


2024-10-07 17:57:34,152 - ras_commander.RasHdf - INFO - Found 1 2D Flow Areas



Example 29: Building Face FacePoints
FacePoints Indexes for the first face of the first 2D Flow Area:
[4 5]
